# 03 · Synastry Hits (SPEC-B-003)

Directional synastry aspects are recomputed and validated against the golden fixture used in the API docs.

In [1]:
from __future__ import annotations

import json
import math
import os
from pathlib import Path
import pandas as pd
import sys

ASTRO_ROOT = Path(os.environ.get('ASTROENGINE_ROOT', '..')).resolve()
DOCS_ROOT = Path(os.environ.get('DOCS_SITE_ROOT', 'docs-site')).resolve()
FIXTURES = DOCS_ROOT / 'docs' / 'fixtures'
os.environ.setdefault('SE_EPHE_PATH', str(ASTRO_ROOT / 'datasets' / 'swisseph_stub'))
if str(ASTRO_ROOT) not in sys.path:
    sys.path.insert(0, str(ASTRO_ROOT))

In [2]:
events = pd.read_csv(FIXTURES / 'birth_events.csv')
subject = {key: events.iloc[0][key] for key in ['ts', 'lat', 'lon']}
partner = {key: events.iloc[1][key] for key in ['ts', 'lat', 'lon']}

In [3]:
from astroengine.synastry.orchestrator import compute_synastry
hits = compute_synastry(subject=subject, partner=partner, aspects=[0, 60, 90, 120, 180], orb_deg=2.0)
print('Hit count:', len(hits))
[vars(hits[0])] if hits else []

Hit count: 18


[{'direction': 'A->B',
  'moving': 'Jupiter',
  'target': 'Pluto',
  'angle_deg': 120.0,
  'orb_abs': 1.2266583829778597,
  'score': 0.38667080851107016,
  'domains': {'SPIRIT': 0.6071428571428572,
   'MIND': 0.14285714285714288,
   'BODY': 0.25}}]

In [4]:
fixture = json.loads((FIXTURES / 'synastry_hits.json').read_text())
calc = [
    {
        'direction': h.direction,
        'moving': h.moving,
        'target': h.target,
        'angle_deg': h.angle_deg,
        'orb_abs': round(h.orb_abs, 6),
    }
    for h in hits
]
fixture[:2], calc[:2]

([{'angle_deg': 120.0,
   'direction': 'A->B',
   'domains': {'BODY': 0.25,
    'MIND': 0.14285714285714288,
    'SPIRIT': 0.6071428571428572},
   'moving': 'Jupiter',
   'orb_abs': 1.2266583829778597,
   'score': 0.38667080851107016,
   'target': 'Pluto'},
  {'angle_deg': 90.0,
   'direction': 'A->B',
   'domains': {'MIND': 0.5, 'SPIRIT': 0.5},
   'moving': 'Neptune',
   'orb_abs': 1.8183528766861627,
   'score': 0.09082356165691863,
   'target': 'Mercury'}],
 [{'direction': 'A->B',
   'moving': 'Jupiter',
   'target': 'Pluto',
   'angle_deg': 120.0,
   'orb_abs': 1.226658},
  {'direction': 'A->B',
   'moving': 'Neptune',
   'target': 'Mercury',
   'angle_deg': 90.0,
   'orb_abs': 1.818353}])

In [5]:
import hashlib
print('Results checksum:', hashlib.sha256((FIXTURES / 'synastry_hits.json').read_bytes()).hexdigest())

Results checksum: 3223a02ff71202e961531f3b19360659c822bbea6f68b2105827c56acf47884c
